In [ ]:
# importing the libraries
import tweepy
import configparser
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Initialising credentials for authorization

config = configparser.ConfigParser()
config.read('config.ini')

consumer_key = config['twitter']['consumer_key']
consumer_secret = config['twitter']['consumer_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [ ]:
# setting the api 
try:
    # authorizing api 
    auth = tweepy.OAuthHandler(consumer_key , consumer_secret)

    # setting access tokens
    auth.set_access_token(access_token , access_token_secret)

    # create api object
    api = tweepy.API(auth , wait_on_rate_limit = True)

except:
    print("Authorization Failed...")    

In [ ]:
#Extracting the tweets from a user
#name = input("Enter the username of person : ")
keyword = input("Enter the keyword or hash or userhandle : ")
limit = int(input("Enter the number of tweets to be extracted : "))
tweets = tweepy.Cursor(api.search_tweets,q = keyword , lang = 'en',count = 100 ,
tweet_mode = 'extended').items(limit)

In [ ]:
# Creating a dataframe
data =[]
for tweet in tweets:
    data.append([tweet.user.screen_name,tweet.full_text])

dafa = pd.DataFrame(data,columns = ['users','tweets'])

dafa.head

In [ ]:
# Function for Cleaning the tweets 
def tweets_cleaner(text):
    text = re.sub(r'@[A-za-z0-9]+','',text)
    text = re.sub(r'#','',text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\s+','',text)
    text = text.lower()
    return text


In [ ]:
# Create a clean tweets string
dafa['tweets'] = dafa['tweets'].apply(tweets_cleaner)

#Display cleaned tweets
dafa.head

In [ ]:
# Creating subjectivity and polarity of tweet
def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getpolarity(text):
    return TextBlob(text).sentiment.polarity

dafa['subjectivity'] = dafa['tweets'].apply(getsubjectivity)
dafa['polarity'] = dafa['tweets'].apply(getpolarity)

dafa

In [ ]:
# creating word cloud
allwords = ' '.join(twts for twts in dafa['tweets'])
wordCloud = WordCloud(width = 500 , height = 300 , random_state = 21 , max_font_size = 110).generate(allwords)

plt.imshow(wordCloud , interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Defining analysis on the basis of polarity
def getanalysis(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'    

dafa['Analysis'] = dafa['polarity'].apply(getanalysis)
dafa

In [ ]:
# Creating a sorted dataframe
sorteddafa = dafa.sort_values(by=['polarity'])

In [ ]:
#printing +ve , -ve , neutral tweets seperately
def positweets():
    j = 1
    for i in range(0 , sorteddafa.shape[0]):
        if (sorteddafa['Analysis'][i] == 'Positive'):
            print(str(j) + ') ' + sorteddafa['tweets'][i])
            print()
            j = j + 1

def negtweets():
    j = 1
    for i in range(0 , sorteddafa.shape[0]):
        if (sorteddafa['Analysis'][i] == 'Negative'):
            print(str(j) + ') ' + sorteddafa['tweets'][i])
            print()
            j = j + 1

def neutweets():
    j = 1
    for i in range(0 , sorteddafa.shape[0]):
        if (sorteddafa['Analysis'][i] == 'Neutral'):
            print(str(j) + ') ' + sorteddafa['tweets'][i])
            print()
            j = j + 1    

In [ ]:
#plot polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range(0,dafa.shape[0]):
    plt.scatter(dafa['polarity'][i] , dafa['subjectivity'][i],color='Blue')
plt.title('Sentiment Analysis') 
plt.xlabel('polarity')
plt.ylabel("subjectivity") 
plt.show()  

In [ ]:
# get percent of possitive negative and neutral tweets 

def pospercent():
    ptweets = dafa[dafa.Analysis == 'Positive']
    ptweets = ptweets['tweets'] 
    print(round((ptweets.shape[0] / dafa.shape[0]) * 100 , 1))

def negpercent():
    ptweets = dafa[dafa.Analysis == 'Negative']
    ptweets = ptweets['tweets'] 
    print(round((ptweets.shape[0] / dafa.shape[0]) * 100 , 1)) 

def neupercent():
    ptweets = dafa[dafa.Analysis == 'Neutral']
    ptweets = ptweets['tweets'] 
    print(round((ptweets.shape[0] / dafa.shape[0]) * 100 , 1))  

pospercent()
negpercent()
neupercent()         

In [ ]:
#show the value count
dafa['Analysis'].value_counts()

In [ ]:
#plot and visualize the count
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('count')
dafa['Analysis'].value_counts().plot(kind = 'bar')
plt.show()